# Dataset

In [ ]:
!pip install pyvi
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from string import digits


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## LOAD DATA
import csv
import pandas as pd


DATA_TRAIN_PATH = '/content/drive/MyDrive/DATH/train.csv'
DATA_TEST_PATH = '/content/drive/MyDrive/DATH/test.csv'
DATA_VAL_PATH = '/content/drive/MyDrive/DATH/val.csv'

test_df = pd.read_csv(DATA_TEST_PATH, sep = ',')
train_df = pd.read_csv(DATA_TRAIN_PATH, sep = ',')
val_df = pd.read_csv(DATA_VAL_PATH, sep = ',')
print(test_df)
print(train_df)
print(val_df)

      Unnamed: 0             ID  \
0              0  2018-En-01559   
1              1  2018-En-03739   
2              2  2018-En-00385   
3              3  2018-En-03001   
4              4  2018-En-01988   
...          ...            ...   
3254         254  2018-En-03848   
3255         255  2018-En-00416   
3256         256  2018-En-03717   
3257         257  2018-En-03504   
3258         258  2018-En-00115   

                                                  Tweet  anger  anticipation  \
0     @Adnan__786__ @AsYouNotWish Dont worry Indian ...   True          True   
1     Academy of Sciences, eschews the normally sobe...  False         False   
2                     I blew that opportunity -__- #mad   True         False   
3                This time in 2 weeks I will be 30... 😥  False         False   
4     #Deppression is real. Partners w/ #depressed p...  False         False   
...                                                 ...    ...           ...   
3254  shaft abrasio

In [ ]:
import gc
train_df = pd.concat([train_df, val_df], ignore_index=True)
del val_df
gc.collect()
print(train_df)

      Unnamed: 0             ID  \
0              0  2017-En-21441   
1              1  2017-En-31535   
2              2  2017-En-21068   
3              3  2017-En-31436   
4              4  2017-En-22195   
...          ...            ...   
7719         881  2018-En-01993   
7720         882  2018-En-01784   
7721         883  2018-En-04047   
7722         884  2018-En-03041   
7723         885  2018-En-03386   

                                                  Tweet  anger  anticipation  \
0     “Worry is a down payment on a problem you may ...  False          True   
1     Whatever you decide to do make sure it makes y...  False         False   
2     @Max_Kellerman  it also helps that the majorit...   True         False   
3     Accept the challenges so that you can literall...  False         False   
4     My roommate: it's okay that we can't spell bec...   True         False   
...                                                 ...    ...           ...   
7719  @BadHombreNPS

In [ ]:
for i, column in enumerate(test_df):
  if(i>=3):
    test_df[column] = test_df[column].astype(int)
for i, column in enumerate(train_df):
  if(i>=3):
    train_df[column] = train_df[column].astype(int)

In [ ]:
# from datasets import Dataset
# test_set = Dataset.from_pandas(test_df)

In [ ]:
# print(test_set)

In [ ]:
# for data in test_dataloader:
  # print( data)

In [ ]:
from nltk.tokenize import sent_tokenize
# sents = sent_tokenize(train_set[30]['Data'])
def preprocess(sent):
  sents = sent_tokenize(sent)
  sent = [a.replace('-', '') for a in sent]
  sent = ''.join(sent)
  return sent

In [ ]:

print([preprocess(r.lower()) for r in test_df['Tweet']])

['@adnan__786__ @asyounotwish dont worry indian army is on its ways to dispatch all terrorists to hell', 'academy of sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation', 'i blew that opportunity __ #mad', 'this time in 2 weeks i will be 30... 😥', '#deppression is real. partners w/ #depressed people truly dont understand the depth in which they affect us. add in #anxiety &amp;makes it worse', '@politicoeurope interesting choice of words... are you confirming that governments fund #terrorism? bit of an open door, but still...', '@ananavarro cnn should, for sure 😀', 'distance yourself once stretched by your friends impose! #serious #loveyou #notseriously', 'be happy. be confident. be kind.\\n\\n #kissableslovesmshopmag\\nalloutdenimfor kissmarc', 'my visit to hospital for care triggered #trauma from accident 20+yrs ago and image of my dead brother in it. feeling symptoms of #depression', "thanks to dollis hill's

In [ ]:
# ## Performing segmentation
# from pyvi.ViTokenizer import tokenize
test_df.loc[:, 'Tweet'] = [preprocess(r.lower()) for r in test_df['Tweet']]
train_df.loc[:, 'Tweet'] = [preprocess(r.lower()) for r in train_df['Tweet']]

In [ ]:
column_name = test_df.columns.tolist()
labels = column_name[3:]
print(labels)
label2id = {k:v for v,k in enumerate(labels)}
id2label = {v:k for v,k in enumerate(labels)}
print(label2id)
print(id2label)

['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
{'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'love': 5, 'optimism': 6, 'pessimism': 7, 'sadness': 8, 'surprise': 9, 'trust': 10}
{0: 'anger', 1: 'anticipation', 2: 'disgust', 3: 'fear', 4: 'joy', 5: 'love', 6: 'optimism', 7: 'pessimism', 8: 'sadness', 9: 'surprise', 10: 'trust'}


In [ ]:
def prepare_data(df):
  rows_list = []

  for i in range(len(df)):
      dict1 = {}
      anewlist = df.iloc[i, 3:].tolist()
      cache = [i for i, x in enumerate(anewlist) if x]
      anewlist = cache + [-1]*(len(anewlist) - len(cache))
      dict1.update({"Tweet": df.iloc[i, 2], "Class": anewlist})

      rows_list.append(dict1)
  # print(rows_list)
  df = pd.DataFrame(rows_list)
  return df

In [ ]:
test_df = prepare_data(test_df)
train_df = prepare_data(train_df)

In [ ]:
print(test_df)
print(train_df)

                                                  Tweet  \
0     @adnan__786__ @asyounotwish dont worry indian ...   
1     academy of sciences, eschews the normally sobe...   
2                       i blew that opportunity __ #mad   
3                this time in 2 weeks i will be 30... 😥   
4     #deppression is real. partners w/ #depressed p...   
...                                                 ...   
3254  shaft abrasions from panties merely shifted to...   
3255  @lomadia heard of remothered? indie horror gam...   
3256        all this fake outrage. y'all need to stop 🤣   
3257  would be ever so grateful if you could record ...   
3258  i'm the wholesome drunk that sends people meme...   

                                             Class  
0        [0, 1, 6, 10, -1, -1, -1, -1, -1, -1, -1]  
1      [2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]  
2        [0, 2, 8, -1, -1, -1, -1, -1, -1, -1, -1]  
3       [4, 8, -1, -1, -1, -1, -1, -1, -1, -1, -1]  
4       [3, 8, -1, -1, -1,

In [ ]:
from datasets import Dataset
test_set = Dataset.from_pandas(test_df)
train_set = Dataset.from_pandas(train_df)


## Sent Embeddings


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer


PhobertTokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model.to('cuda')

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
def tokenize_function_sent(example):
  input = PhobertTokenizer(example['Data'], padding=True, truncation=True, return_tensors="pt")
  input.to('cuda')
  with torch.no_grad():
    embeddings = model(**input, output_hidden_states=True, return_dict=True).pooler_output
  embeddings = embeddings.to('cpu')
  embeddings = embeddings.numpy()
  list_emb = list(embeddings)
  example['Data'] = list_emb
  return example


tokenized_train_datasets = train_set.map(tokenize_function_sent, batched=True, batch_size = 64)
tokenized_test_datasets = test_set.map(tokenize_function_sent, batched=True, batch_size = 64)


Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_datasets)

Dataset({
    features: ['Class', 'Data'],
    num_rows: 5100
})


In [ ]:
tokenized_train_datasets.set_format("torch")
tokenized_test_datasets.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_train_datasets, batch_size=64, shuffle=True)
test_dataloader = DataLoader(tokenized_test_datasets, batch_size=64, shuffle=True)

In [ ]:
print((tokenized_train_datasets["Class"].shape))
print((tokenized_train_datasets["Data"].shape))

torch.Size([5100])
torch.Size([5100, 768])


## Word Embeddings

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

phobert = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function_word(example):
  max_len = 128
  sentences = example['Tweet']
  input_ids = [tokenizer.encode(sentence , max_length = max_len, truncation = True) for sentence in sentences]
  phobert.cuda()
  with torch.no_grad():
      features = [phobert(torch.tensor([input_id]).to('cuda')) for input_id in input_ids]  # Models outputs are now tuples

  seqs = [torch.squeeze(feature.last_hidden_state) for feature in features]

  seqs[0] = torch.permute(seqs[0], (1, 0))
  seqs[0] = nn.ConstantPad1d((0, max_len - seqs[0].shape[1]), 0)(seqs[0])
  seqs[0] = torch.permute(seqs[0], (1, 0))

  embeddings = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True)
  embeddings = embeddings.to('cpu')
  embeddings = embeddings.numpy()

  list_emb = list(embeddings)
  example['Tweet'] = list_emb

  return example


In [ ]:
# def tokenize_function_word_normal(example):
#   # max_len = 256
#   # sentences = example['Data']
#   input_ids = tokenizer(example["Data"], max_length = 256, padding = "max_length", truncation = True, return_tensors = 'pt')
#   input_ids = input_ids.to('cuda')
#   phobert.cuda()
#   with torch.no_grad():
#       features = phobert(**input_ids) # Models outputs are now tuples
#   embeddings = features[0]
#   embeddings = embeddings.to('cpu')
#   embeddings = embeddings.numpy()

#   list_emb = list(embeddings)
#   example['Data'] = list_emb

#   return example


In [ ]:
del test_df, train_df
gc.collect()

108

In [ ]:
tokenized_test_datasets = test_set.map(tokenize_function_word, batched=True, batch_size = 256)
del test_set
gc.collect()
tokenized_train_datasets = train_set.map(tokenize_function_word, batched=True, batch_size = 256)
del train_set
gc.collect()

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

Map:   0%|          | 0/7724 [00:00<?, ? examples/s]

30

In [ ]:
# tokenized_train_datasets = train_set.map(tokenize_function_word, batched=True, batch_size = 128)
# del train_set
# gc.collect()

In [ ]:
# print(tokenized_train_datasets['Class'])

In [ ]:
from torch.utils.data import DataLoader
tokenized_train_datasets.set_format("torch")
tokenized_test_datasets.set_format("torch")



In [ ]:
# test_dataloader = DataLoader(tokenized_test_datasets, batch_size=4, shuffle=True)
# for batch, data in enumerate(tqdm(test_dataloader)):
#     # X = torch.permute(data["Tweet"], (0,2,1))
#     # print(X.shape)
#     y = data['Class']
#     print(y.shape)
#     print(y)


Flow: sent_tokenize + segmentation + tokenizer + embedding

Apply method (CNN, RNN, LSTM)

# Define network and Train

## WANDB setup

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
# !wandb login

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project='sentiment_analysis_project_DATH',

    # track hyperparameters and run metadata
    config={
    'in_features': 768,
    'kernel_sizes': [3,4,5],
    # 'num_filters': [512,128],
    'drop': 0.2,
    'num_filters': 256,
    'seq_len': 256,
    "learning_rate": 0.001,
    "architecture": "CNN",
    "epochs": 10,
    # "cell": 2
    "embedding_method": 'normal',
    "activation": "RELU"
    }
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Embedding, Dropout,concatenate
# from tensorflow.keras.layers import Reshape, Flatten
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model
# from tensorflow.keras import regularizers
# sequence_length = data.shape[1]
# filter_sizes = [3,4,5]
# num_filters = 100
# drop = 0.5

# inputs = Input(shape=(sequence_length,))
# embedding = embedding_layer(inputs)
# # reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

# conv_0 = Conv1D(num_filters, filter_sizes[0],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
# conv_1 = Conv1D(num_filters, filter_sizes[1],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
# conv_2 = Conv1D(num_filters, filter_sizes[2],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
# print(conv_1)
# maxpool_0 = MaxPooling1D(sequence_length - filter_sizes[0] + 1, strides=1)(conv_0)
# maxpool_1 = MaxPooling1D(sequence_length - filter_sizes[1] + 1, strides=1)(conv_1)
# maxpool_2 = MaxPooling1D(sequence_length - filter_sizes[2] + 1, strides=1)(conv_2)

# merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
# flatten = Flatten()(merged_tensor)
# reshape = Reshape((3*num_filters,))(flatten)
# dropout = Dropout(drop)(flatten)
# output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)


# # this creates a model that includes
# model = Model(inputs, output)


# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# model.summary()

# #define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
# callbacks_list = [early_stopping]


In [ ]:
# for i, data in enumerate(tqdm(train_dataloader)):
#   input = torch.permute(data["Data"], (0,2,1))
#   output = ngramModel(input)
#   print(output.shape)
#   # print(output)

In [ ]:
import torch.optim as optim
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

In [ ]:
# a = torch.rand(2, 3)
# b = torch.rand(2, 3)
# print(a)
# print(b)
# print(a[0][1])

In [ ]:
# a = torch.tensor([8, 8, 8, 8, 8, 4])
# b = torch.tensor([[1, 0, 1, 1, 1, 0, 0,0,0,0, 0],
#                   [0, 0, 1, 0, 1, 0, 0,0,1,0, 0],
#                   [1, 1, 1, 0, 1, 0, 0,0,0,0, 0],
#                   [1, 0, 0, 1, 0, 0, 0,0,1,0, 0],
#                   [0, 0, 1, 1, 1, 0, 0,0,0,1, 0],
#                   [1, 0, 1, 0, 1, 0, 0,0,0,1, 0],
#                   ])

In [ ]:
def calculate_accuracy(logits, label):
  correct = 0
  for i, x in enumerate(label):
    el = logits[i]
    # print(el)
    if(el in x):
      correct +=1

  # print(logits, label)
  acc = correct/len(label)
  return acc


In [ ]:
# calculate_accuracy(a,b)

In [ ]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, data in enumerate(tqdm(dataloader)):
      # Send data to target device

      ##FIXXX THIS
      X = torch.permute(data["Tweet"], (0,2,1))
      y = data['Class']

      ##########

      X, y = X.to(device), y.to(device)
      model.to(device)
      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(y_pred, dim=1)
      # print(y_pred)
      # print((y_pred>0.5).type(torch.uint8)   )
      # print(y_pred_class)
      # print(y)
      train_acc += calculate_accuracy(y_pred_class, y)
      # train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, data in enumerate(tqdm(dataloader)):
          X = torch.permute(data["Tweet"], (0,2,1))
          y = data['Class']

          # Send data to target device
          X, y = X.to(device), y.to(device)
          model.to(device)
          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          # print(test_pred_logits)
          # print(test_pred_labels)
          # print(y)
          test_acc += calculate_accuracy(test_pred_labels, y)
          # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [ ]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }
  best_accuracy = 0
  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

      train_metrics = {'train/train_loss': train_loss,
                       'train/train_accuracy':train_acc }
      val_metrics = {"test/test_loss": test_loss,
                "test/test_accuracy": test_acc}
      if  test_acc > best_accuracy:
        torch.save(model.state_dict(), 'best-model-parameters.pt')
      wandb.log({**train_metrics, **val_metrics})
  # Return the filled results at the end of the epochs
  return results

In [ ]:
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  model.load_state_dict(torch.load("best-model-parameters.pt"))
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

## CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
in_features = 768
kernel_sizes = [3,4,5]
num_filters = wandb.config.num_filters
drop = wandb.config.drop
seq_len = 128
class CNNnet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_0 = nn.Conv1d(in_features, num_filters, kernel_sizes[0])
    self.pool_0 = nn.MaxPool1d(seq_len - kernel_sizes[0] + 1, stride = 1)

    self.conv_1 = nn.Conv1d(in_features, num_filters, kernel_sizes[1])
    self.pool_1 = nn.MaxPool1d(seq_len - kernel_sizes[1] + 1, stride = 1)

    self.conv_2 = nn.Conv1d(in_features, num_filters, kernel_sizes[2])
    self.pool_2 = nn.MaxPool1d(seq_len - kernel_sizes[2] + 1 , stride = 1)

    self.flatten = nn.Flatten()

    self.dropout = nn.Dropout(drop)

    self.dense_0 = nn.Linear(num_filters * 3, 11)

  def forward(self, x):
    x_0 = F.relu(self.pool_0(self.conv_0(x)))
    x_1 = F.relu(self.pool_1(self.conv_1(x)))
    x_2 = F.relu(self.pool_2(self.conv_2(x)))
    merged_tensor = self.flatten(torch.cat([x_0, x_1, x_2], dim = 1))
    merged_tensor = self.dropout(merged_tensor)
    x = F.relu(self.dense_0(merged_tensor))
    return x


In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
LEARNING_RATE = 0.001

ngramModel = CNNnet()

# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.MultiLabelMarginLoss()
optimizer = optim.Adam(ngramModel.parameters(), lr=LEARNING_RATE)
train_dataloader = DataLoader(tokenized_train_datasets, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(tokenized_test_datasets, batch_size=BATCH_SIZE, shuffle=True)
results = train(model=ngramModel,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device='cuda')

wandb.finish()
save_model(model=ngramModel,
                 target_dir="/content/drive/MyDrive/NLP_Assignment/model",
                 model_name=f"CNNnet_{NUM_EPOCHS}epoch_{num_filters}_{drop}_embedding_normal.pth")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1439 | train_acc: 0.5805 | test_loss: 0.8303 | test_acc: 0.7351


  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch: 10 | train_loss: 0.4431 | train_acc: 0.9172 | test_loss: 0.7222 | test_acc: 0.8056


test/test_accuracy,█▁▂▃▃▃▃▃▃▃▃▄
test/test_loss,██▆▃▃▁▁▁▁▁▂▁
train/train_accuracy,█▁▁▄▅▅▆▆▆▆▇▇
train/train_loss,▇██▄▃▃▂▂▂▁▁▁
test/test_accuracy,0.80563
test/test_loss,0.72224
train/train_accuracy,0.91721
train/train_loss,0.44312


[INFO] Saving model to: /content/drive/MyDrive/NLP_Assignment/model/CNNnet_10epoch_256_0.2_embedding_normal.pth


## LSTM

In [ ]:
###OLD CODE
# import torch.nn as nn
# import torch.nn.functional as F
# in_features = 768
# kernel_sizes = [3,4,5]
# hidden_size = 128
# drop = 0.5
# seq_len = 256
# class LSTMnet(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.lstm = nn.LSTM(input_size = in_features, hidden_size = hidden_size, num_layers = 2, bidirectional  = True   ,batch_first= True , dropout = drop)
#     self.dense_0 = nn.Linear(hidden_size, 3)
#   def forward(self, x):
#     x,  (hidden, _)= self.lstm(x)
#     hidden = hidden[-1]
#     x =  F.softmax(self.dense_0(hidden))
#     return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
in_features = 768
kernel_sizes = [3,4,5]
hidden_size = 128
hidden_size_2 = 128
drop = 0
seq_len = 128
class LSTMnet(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm_0 = nn.LSTMCell(input_size = in_features, hidden_size = hidden_size)
    self.lstm_1 = nn.LSTMCell(input_size = hidden_size, hidden_size = hidden_size_2)
    self.dropout = nn.Dropout(drop)
    self.dense_0 = nn.Linear(hidden_size_2, 11)
  def forward(self, x):
    x = torch.permute(x, (2,0,1))
    output = []
    for i in range(x.size()[0]):
        hx, cx = self.lstm_0(x[i])
        output.append(hx)
    x = torch.stack(output, dim=0) #256, 4, 512
    for i in range(x.size()[0]):
        hx, cx = self.lstm_1(x[i])
        output.append(hx)
    # print(output)
    x = torch.stack(output, dim=0) #256, 4, 128
    # x = torch.permute(x, (1,))
    x = torch.mean(x, dim = 0)
    # x = self.dropout(x)
    x =  F.relu(self.dense_0(x))
    # print(x.shape)
    return x

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
LEARNING_RATE = 0.001

LSTMmodel = LSTMnet()

loss_fn = nn.MultiLabelMarginLoss()
optimizer = optim.Adam(LSTMmodel.parameters(), lr=LEARNING_RATE)
train_dataloader = DataLoader(tokenized_train_datasets, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(tokenized_test_datasets, batch_size=BATCH_SIZE, shuffle=True)
results = train(model=LSTMmodel,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device='cuda')

wandb.finish()
save_model(model=LSTMmodel,
                 target_dir="/content/drive/MyDrive/NLP_Assignment/model",
                 model_name=f"LSTMmodel_{NUM_EPOCHS}epoch_{hidden_size}_{drop}_2cell.pth")

## LSTM + CNN

# Proposed Pre-processing

In [ ]:
## SEGMENTATION
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation", max_length=512, truncation = True, padding='max_length')
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")
word_segmentation_model = pipeline("token-classification", model=model, tokenizer=tokenizer)
def word_segmentation(text):
  ner_results = word_segmentation_model(text)
  example_tok = ""
  for e in ner_results:
      if "##" in e["word"]:
          example_tok = example_tok + e["word"].replace("##","")
      elif e["entity"] =="I":
          example_tok = example_tok + "_" + e["word"]
      else:
          example_tok = example_tok + " " + e["word"]
  return example_tok

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
sents = sent_tokenize(train_set[30]['Data'])
print(sents)
for sent in sents:
   print(word_segmentation(sent))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['Không biết Apple làm vậy để làm gì, họ thu lợi quá nhiều từ chiếc điện thoại lại còn cố móc túi người dùng thêm nữa từ sự chênh lệch về dung lượng.']
 Không biết Apple làm vậy để làm gì , họ thu lợi quá nhiều từ chiếc điện_thoại lại còn cố móc_túi người dùng thêm nữa từ sự chênh_lệch về dung_lượng .


In [ ]:
new_word = word_segmentation(train_set[20]['Data'])
print(sent_tokenize(new_word))

[' ai cũng chê iphone nhưng cuối_cùng cũng SS bắt trước iphone vè màu_sắc , thiết_kế jack 3 .', '5 phía dưới , loa có lỗ nhỏ , đường viền dải anten']
